In [1]:
import json
import sqlite3
import pandas as pd

In [2]:
def get_lap_time_statistics(conn, drivers, year_exclusion=2023):
    max_query = f'''
        SELECT
            drivers.driverId,
            strftime('%Y', races.date) AS season,
            MAX(lap_times.milliseconds) AS max_lap_time
        FROM
            lap_times
        JOIN
            races ON lap_times.raceId = races.raceId
        JOIN
            drivers ON lap_times.driverId = drivers.driverId
        WHERE
            strftime('%Y', races.date) != '{year_exclusion}'
        GROUP BY
            drivers.driverId, season
    '''

    avg_query = f'''
        SELECT
            drivers.driverId,
            strftime('%Y', races.date) AS season,
            AVG(lap_times.milliseconds) AS avg_lap_time
        FROM
            lap_times
        JOIN
            races ON lap_times.raceId = races.raceId
        JOIN
            drivers ON lap_times.driverId = drivers.driverId
        WHERE
            strftime('%Y', races.date) != '{year_exclusion}'
        GROUP BY
            drivers.driverId, season
    '''

    min_query = f'''
        SELECT
            drivers.driverId,
            strftime('%Y', races.date) AS season,
            MIN(lap_times.milliseconds) AS min_lap_time
        FROM
            lap_times
        JOIN
            races ON lap_times.raceId = races.raceId
        JOIN
            drivers ON lap_times.driverId = drivers.driverId
        WHERE
            strftime('%Y', races.date) != '{year_exclusion}'
        GROUP BY
            drivers.driverId, season
    '''

    max_lap_times_df = pd.read_sql_query(max_query, conn)
    avg_lap_times_df = pd.read_sql_query(avg_query, conn)
    min_lap_times_df = pd.read_sql_query(min_query, conn)

    max_lap_times_df['max_lap_times_json'] = max_lap_times_df.apply(
        lambda row: json.dumps({row['season']: row['max_lap_time']}), axis=1
    )

    avg_lap_times_df['avg_lap_times_json'] = avg_lap_times_df.apply(
        lambda row: json.dumps({row['season']: row['avg_lap_time']}), axis=1
    )

    min_lap_times_df['min_lap_times_json'] = min_lap_times_df.apply(
        lambda row: json.dumps({row['season']: row['min_lap_time']}), axis=1
    )

    max_lap_times_json_df = max_lap_times_df.groupby('driverId')['max_lap_times_json'].apply(list).reset_index()
    avg_lap_times_json_df = avg_lap_times_df.groupby('driverId')['avg_lap_times_json'].apply(list).reset_index()
    min_lap_times_json_df = min_lap_times_df.groupby('driverId')['min_lap_times_json'].apply(list).reset_index()

    drivers = pd.merge(drivers, max_lap_times_json_df, on='driverId', how='left')
    drivers = pd.merge(drivers, avg_lap_times_json_df, on='driverId', how='left')
    drivers = pd.merge(drivers, min_lap_times_json_df, on='driverId', how='left')

    drivers.rename(columns={'max_lap_times_json': 'max_lap_times', 'avg_lap_times_json': 'avg_lap_times', 'min_lap_times_json': 'min_lap_times'}, inplace=True)

    del min_lap_times_json_df, max_lap_times_json_df, avg_lap_times_json_df
    
    print(drivers)

    return drivers



In [3]:
def get_lap_positions(conn, existing_df):
    query = '''
    WITH RankedPositions AS (
        SELECT
            d.driverId,
            r.year,
            lt.position AS position,
            COUNT(lt.position) AS position_count,
            ROW_NUMBER() OVER (PARTITION BY d.driverId, r.year ORDER BY COUNT(lt.position) DESC) AS position_rank,
            COUNT(lt.position) OVER (PARTITION BY d.driverId, r.year) AS total_races
        FROM
            drivers d
        JOIN
            lap_times lt ON d.driverId = lt.driverId
        JOIN
            races r ON lt.raceId = r.raceId
        WHERE
            r.year != 2023 
        GROUP BY
            d.driverId, r.year, lt.position
    )
    SELECT
        driverId,
        year,
        position,
        position_count
    FROM
        RankedPositions
    WHERE
        position_rank <= 3;
    '''

    df = pd.read_sql_query(query, conn)

    lap_positions_json_df = df.groupby('driverId').apply(
        lambda group: json.dumps({year: group[['position', 'position_count']].to_dict(orient='records') for year, group in group.groupby('year')})
    ).reset_index(name='lap_positions')

    merged_df = pd.merge(existing_df, lap_positions_json_df, on='driverId', how='left')

    del lap_positions_json_df

    return merged_df

In [4]:
def get_total_laps_count(conn: str) -> pd.DataFrame:
    """
    Get the total number of laps completed by each driver in each year.

    :param conn: The database connection.
    :return: DataFrame containing driverId, year, and total_laps columns.
    """
    total_laps_query = '''
    SELECT
        r.year,
        lt.driverId,
        COUNT(*) AS total_laps
    FROM
        lap_times lt
    JOIN
        races r ON lt.raceId = r.raceId
    WHERE
        r.year != '2023'
    GROUP BY
        lt.driverId, r.year;
    '''
    
    total_laps_df = pd.read_sql_query(total_laps_query, conn)

    return total_laps_df

In [5]:
def get_fastest_slowest_laps(conn):
    query = """
    SELECT
        races.year AS year,
        MIN(lap_times.milliseconds) AS fastest_lap_time,
        MAX(lap_times.milliseconds) AS slowest_lap_time
    FROM
        lap_times
    JOIN
        races ON lap_times.raceId = races.raceId
    GROUP BY
        races.year
    ORDER BY
        races.year;
    """

    result_df = pd.read_sql_query(query, conn)
    return result_df


In [6]:
def get_lap_times_stats(conn: str) -> pd.DataFrame:
    """
    This function executes an SQL query to retrieve lap time statistics for each driver and year,
    including the minimum, average, and maximum percentage difference from the fastest lap.

    :param conn: The database connection.

    :return: A DataFrame containing driverId, year, min_lap_time_diff, avg_lap_time_diff and max_lap_time_diff columns.
    """
    lap_times_query = """
        SELECT
            r.year,
            d.driverId,
            MIN((CAST(lt.milliseconds AS FLOAT) - fl.fastest_lap) / fl.fastest_lap * 100.0) AS min_lap_time_diff,
            AVG((CAST(lt.milliseconds AS FLOAT) - fl.fastest_lap) / fl.fastest_lap * 100.0) AS avg_lap_time_diff,
            MAX((CAST(lt.milliseconds AS FLOAT) - fl.fastest_lap) / fl.fastest_lap * 100.0) AS max_lap_time_diff
        FROM
            lap_times lt
            JOIN drivers d ON lt.driverId = d.driverId
            JOIN races r ON lt.raceId = r.raceId
            JOIN (
                SELECT
                    lt.raceId,
                    MIN(lt.milliseconds) AS fastest_lap
                FROM
                    lap_times lt
                GROUP BY
                    lt.raceId
            ) fl ON lt.raceId = fl.raceId
        WHERE
            r.year != 2023
        GROUP BY
            d.driverId,
            r.year;
    """
    lap_times_stats = pd.read_sql_query(lap_times_query, conn)

    return lap_times_stats

In [9]:
def get_median_lap_position(conn: str) -> pd.DataFrame:
    """
    Calculate the median lap position for each driver and year based on lap times.

    :param conn: The database connection.
    :return: A DataFrame containing driverId, year, and median_lap_position columns.
    """
    median_lap_position_query = '''
        WITH RankedLaps AS (
            SELECT
                r.year,
                lt.driverId,
                lt.position,
                ROW_NUMBER() OVER (PARTITION BY lt.driverId, r.year ORDER BY lt.position) AS row_num,
                COUNT(*) OVER (PARTITION BY lt.driverId, r.year) AS total_count
            FROM
                lap_times lt
            JOIN
                races r ON lt.raceId = r.raceId
            WHERE
                r.date != '2023'
        ),
        MedianCandidates AS (
            SELECT
                driverId,
                year,
                AVG(position) AS median_candidate
            FROM
                RankedLaps
            WHERE
                row_num = (total_count + 1) / 2 OR row_num = (total_count + 2) / 2
            GROUP BY
                driverId, year
        )
        SELECT
            driverId,
            year,
            CAST(AVG(median_candidate) AS INTEGER) AS median_lap_position
        FROM
            MedianCandidates
        GROUP BY
            driverId, year;
    '''
    median_lap_position_df = pd.read_sql_query(median_lap_position_query, conn)

    return median_lap_position_df